# Data Analysis on Public Education Dataset

### 1. Introduction

### 2. Data Wrangling

#### 2.1 Data Description

Institution-level data files for 1996-97 through 2020-21 containing aggregate data for each institution. Includes information on institutional characteristics, enrollment, student aid, costs, and student outcomes.

#### 2.2 Data Exploration

#### 2.3 Data Visualization

### 3. EDA

#### 3.1 Correlation

#### 3.2 

### 4. Data Modeling

### 5. Conclustion